In [1]:
import whisper as ws

model = ws.load_model("base")

#이걸 구해야 할건디,,,,
audio = ws.load_audio("test2.m4a")
audio = ws.pad_or_trim(audio)

mel = ws.log_mel_spectrogram(audio).to(model.device)

_, probs = model.detect_language(mel)
print(f'Detected language: {max(probs, key=probs.get)}')

options = ws.DecodingOptions()
result = ws.decode(model, mel, options)

print(result.text)

C:\Users\speed\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\whisper\timing.py:58: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def backtrace(trace: np.ndarray):


Detected language: en
Hello everyone, my name is Kim Hyun-won. We have to do some project. Yeah, this is a sample MP3 file for above project.


In [1]:
#CUDA device detection test
import os
import torch

device_count = torch.cuda.device_count()

for i in range(device_count):
    device = torch.device(f"cuda:{i}")
    print(f"Device {i}: {torch.cuda.get_device_name(device)}")
    print(torch.cuda.get_device_properties(device).total_memory / 1024**3, ' GB')
    print(torch.cuda.get_device_properties(device).total_memory / 1024**3, ' GB')

In [1]:
# VRAM stress test
import torch
import torchvision.models as models

# load ResNet-152 model
model = models.resnet152(pretrained=True)

# move model into GPU
device = torch.device("cuda")
model = model.to(device)

# set large batch size
batch_size = 32

# generate input data
input_data = torch.randn(batch_size, 3, 224, 224).to(device)

# run
output = model(input_data)

# output
print("GPU 메모리 사용량 (bytes):", torch.cuda.max_memory_allocated(device=device))


C:\Users\speed\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\speed\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet152_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet152_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


GPU 메모리 사용량 (bytes): 5967801856


In [1]:
# performance test - GPU mode
import whisper as ws
import torch, time, platform, psutil, extract

model = ws.load_model("medium")

def get_device_info():
    print(f'CPU: {platform.processor()}')
    print(f'Processor count: {psutil.cpu_count(logical=False)}')
    print(f'Memory: {psutil.virtual_memory().total // (1024**3)} GB')
    
    device_count = torch.cuda.device_count()

    for i in range(device_count):
        device = torch.device(f'cuda:{i}')
        print(f'CUDA Device {i}: {torch.cuda.get_device_name(device)}')

def start(path):
    # 오디오 불러오기 및 trim 진행
    audio = ws.load_audio(path)
    audio = ws.pad_or_trim(audio)

    mel = ws.log_mel_spectrogram(audio).to(model.device)

    _, probs = model.detect_language(mel)
    print(f'Detected language: {max(probs, key=probs.get)}')

    options = ws.DecodingOptions(fp16 = False)
    # result = ws.decode(model, mel, options)
    result = model.transcribe(audio)

    #print(type(result))
    # print(result['segments'])
    
    ret = []
    for data in result['segments']:
        start_time = round(data['start'], 2)
        end_time = round(data['end'], 2)
        text = data['text']
        ret.append([start_time, end_time, text])
        
        print(f'{start_time} ~ {end_time}: {text}')
        
    return ret

#run task
get_device_info()
extract.extract()
start_t = time.time()
start('audio.mp3')
end_t = time.time()
print(f'{end_t - start_t:.5f} sec')

c:\Users\speed\anaconda3\envs\shouter\lib\site-packages\whisper\timing.py:58: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def backtrace(trace: np.ndarray):


CPU: Intel64 Family 6 Model 154 Stepping 3, GenuineIntel
Processor count: 12
Memory: 15 GB
CUDA Device 0: NVIDIA GeForce MX550
MoviePy - Writing audio in C:\Code\shouter\shouter\shouter\test\audio.mp3


MoviePy - Done.
Detected language: ko


In [5]:
# performance test - CPU mode
import platform, psutil, time
import whisper as ws
import extract

model = ws.load_model("medium")

def get_device_info():
    print(f'CPU: {platform.processor()}')
    print(f'Processor count: {psutil.cpu_count(logical=False)}')
    print(f'Memory: {psutil.virtual_memory().total // (1024**3)} GB')

def start(path):
    #모델 불러오기
    global model
    
    # 오디오 불러오기 및 trim 진행
    audio = ws.load_audio(path)
    audio = ws.pad_or_trim(audio)

    mel = ws.log_mel_spectrogram(audio).to(model.device)

    _, probs = model.detect_language(mel)
    print(f'Detected language: {max(probs, key=probs.get)}')

    options = ws.DecodingOptions(fp16 = False)
    # result = ws.decode(model, mel, options)
    result = model.transcribe(audio)

    #print(type(result))
    # print(result['segments'])
    
    ret = []
    for data in result['segments']:
        start_time = round(data['start'], 2)
        end_time = round(data['end'], 2)
        text = data['text']
        ret.append([start_time, end_time, text])
        
        print(f'{start_time} ~ {end_time}: {text}')
        
    return ret

#run task
get_device_info()
extract.extract()
start_t = time.time()
start('audio.mp3')
end_t = time.time()
print(f'{end_t - start_t:.5f} sec')

RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
